# Natasha 2

## Постановка задачи

Исследуется задача
$$\min_{x \in \mathbb{R}^d} f(x) = \frac{1}{n} \sum_{i = 1}^n f_i(x) = \mathbb{E}_i[f_i(x)],$$
где $\{f_i(x)\}_{i=1}^n$ - в общем случае гладкие невыпуклые функции 

Необходимо найти точку $\hat{x} \in \mathbb{R}^d$, такую что $\Vert \nabla f(\hat{x}) \Vert \leq \varepsilon$ и $ \nabla^2 f(\hat{x}) \succeq -\delta\mathbf{I} $

## Предположения о виде функции

<ul>
<li> $f$ имеет ограниченную дисперсию, откуда $\mathbb{E}_i  [ \Vert \nabla f_i(x) - \nabla f(x) \Vert ] < \nu$ </li>

<li> $L$-Липшицевость градиента: $\Vert \nabla f(x) - \nabla f(y) \Vert \leq L \cdot \Vert x -y \Vert$ </li>

<!-- 	%\item все собственные значения $\nabla^2 f(x)$ лежат на отрезке $[-\sigma, L]$ -->
<li>$L_2$-Липшицевость гессиана: $\Vert \nabla^2 f(x) - \nabla^2 f(y) \Vert \leq L_2 \cdot \Vert x -y \Vert$</li>
</ul>

## Описание алгоритма
Основная идея: избегать седловых точек, используя для этого информацию из гессиана

На каждой итерации в точке $y_k$ сравниваем минимальное собственное значение $\lambda_{min}(\nabla^2f(y_k))$ и $- \delta$.
<ul>
<li> Если $\exists v : v^T \nabla^2f(y_k) v \leq -\frac{\delta}{2} $, то пытаемся отдалиться от седла: $y_{k+1} = y_k \pm \frac{\delta}{L_2}v$ ($+$ или $-$ выбираем случайно).
Для поиска $v$ пользуемся алгоримом Oja с $\Theta(\frac{L^2}{\delta^2} \log(dk))$ итерациями
</li>
		
<li> Иначе берём $F^k(x) = f(x) + L(\max\{ 0, \Vert x - y_k \Vert - \frac{\delta}{L_2} \})^2$, которая явялется $5L$-Липшицевой по градиенту и $3\delta$-невыпуклой ($\nabla^2 F^k(x) \succeq -3\delta \mathbf{I}$); ищем для неё точку $\widehat{x} : \Vert \nabla F^k(\hat{x}) \Vert \leq \varepsilon$ и переходим туда: $y_{k+1} = \hat{x}$. Как видно, $F^k$ штрафует за выход из безопасной зоны $\{x : \Vert x - y_k \Vert \leq \frac{\delta}{L_2} \}$.
Для поиска $\hat{x}$ запускаем Natasha 1.5 на одну эпоху
</li>
</ul>

Цикл обрывается, когда было совершено $N_1$ шагов первого порядка, и алгоритм выдает последнюю рассмотренную точку $y_k$

Псевдокод: <img src = "pseudocodes/Natasha-2.jpg"/>

## Составляющие

### Natasha 1.5 : Поиск стационарных точек для $\sigma$-невыпуклых функций

Каждая эпоха делится на $p$ суб-эпох со стартовым вектором $\widehat{x}$.

При подсчёте градиентов, вместо $f(x)$ фактически используется $f(x) + \sigma \Vert x - \widehat{x} \Vert^2$, что должно стабилизировать алгоритм, немного замедляя его.
Вместе с этим, используется пересчёт градиента из алгоритма SVRG
Алгоритм выдает последнюю точку $\hat{x}$

Псевдокод : 
<img src = "pseudocodes/Natasha-15.jpg"/>
(в нашем случае $\psi(x) \equiv 0$, поэтому строчка 9 будет выглядеть как $x_{t+1} = x_t - \alpha \tilde{\nabla}$)

### Oja's algorithm : быстрый поиск максимального собственного вектора

Пусть имеется некоторое распределение $\mathcal{D}$ симметричных матриц, собственные числа которых лежат на $[0,1]$ и $\mathbf{B} = \mathbb{E}_{\mathbf{A} \sim \mathcal{D}}[\mathbf{A}]$ - среднее. Пусть $\mathbf{A}_1, \dots \mathbf{A}_T \sim \mathcal{D}$.
Алгоритм начинает со случайного гауссовского вектора $w_1 \in \mathbb{R}^d$ единичной нормы. Затем на каждой итерации $k = 2, \dots T$ подсчитывается $w_k = \frac{1}{C}(\mathbf{I} + \eta \mathbf{A}_{k-1})w_{k-1}$, так чтобы $\Vert w_k \Vert = 1$. Параметр $\eta$ выбирается, так чтобы $\eta = \Theta(\sqrt{T}) $ Алгоритм выдаёт случайный вектор из равномерного распределения $\{ w_1, \dots w_T \}$

В нашем случае $\mathcal{D} = \{ -\frac{1}{L}\nabla^2f_i(x) \}_i$, а итерации выглядят как 
$$ w_k = \frac{1}{CL}(L w_{k-1} - \eta \nabla^2f_k(x)w_{k-1})$$